In [1]:
import nltk
import random
from nltk.corpus import movie_reviews
from nltk.classify.scikitlearn import SklearnClassifier
from nltk import word_tokenize
import pickle

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

from nltk.classify import ClassifierI
from statistics import mode

In [2]:
class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        
        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf
        

In [44]:
import codecs
short_pos = codecs.open("short_reviews/positive.txt","r",encoding="latin2").read()
short_neg = codecs.open("short_reviews/negative.txt","r", encoding="latin2").read()

short_pos = open("pos_tweets_415.txt","r", encoding="ISO-8859-1").read()
short_neg = open("neg_tweets_415.txt","r", encoding="ISO-8859-1").read()

In [3]:
short_pos = open("pos_tweets_415.txt","r", encoding="ISO-8859-1").read()
short_neg = open("neg_tweets_415.txt","r", encoding="ISO-8859-1").read()

In [54]:
pos_lines = [line.rstrip('\n') for line in open('pos_tweets_415.txt', 'r', encoding='ISO-8859-1')]
neg_lines = [line.rstrip('\n') for line in open('neg_tweets_415.txt', 'r', encoding='ISO-8859-1')]

'ï»¿And #bitcoin  is the one and only. #bitcoincash  and #BitcoinSV  are both #shitcoins\n# XLM  Buy at #KuCoin  and sell at #Bitexen . Ratio: 0.78% Buy at #KuCoin  and sell at #EXMO . Ratio: 3.01% Buy at #Bitexen  and sell at #EXMO . Ratio: 1.15% Buy at #HitBTC  and sell at #EXMO . Ratio: 3.16% #bitcoin  #arbitrage  #arbitraj  #arbingtool    http:// arbing.info Ã\x82Â\xa0\n# Crypto  #bitcoin  would take over. That\'s actually the thesis behind the technology\nBoost Your Earnings Up To 8X With Our New CryptoTab Browser #bitcoin  #btc  #cryptocurrency  #googlechrome  #firefox   http:// bit.ly/Btc-Crypto-Tab Ã\x82Â\xa0\nPeople don\'t believe how cool it is in making money from  bitcoin  investment until it is too late. Today\'s millionaires only became one because they took risk. Join the risk takers and ride your way towards financial freedom. Slide into my DM for more details on how to earn more..\nBest  bitcoin  double up investment . Invest $300-earn $600 Invest $500-earn $1.000 Inve

documents = []

for r in pos.split("\n"):
    documents.append((r,"pos"))

for r in neg.split("\n"):
    documents.append((r,"neg"))

In [55]:
all_words = []
documents = []

In [58]:
# to select only adjects = J, adverbs = R and verb = V we delted all the other words in the text.

allowed_word_types = ["J"]

for p in pos_lines:
    documents.append((p,"pos"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

for p in neg_lines:
    documents.append((p,"neg"))
    words = word_tokenize(p)
    pos = nltk.pos_tag(words)
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())

In [59]:
all_words = nltk.FreqDist(all_words)
print(all_words.most_common(100))
print(all_words["bitcoin"])

[('//', 410), ('crypto', 72), ('bitcoin', 54), ('btc', 36), ('@', 24), ('more', 23), ('top', 22), ('last', 19), ('new', 17), ('eth', 16), ('good', 15), ('other', 13), ('digital', 11), ('ã\x82â', 11), ('litecoin', 11), ('next', 11), ('financial', 10), ('most', 10), ('first', 10), ('xrp', 10), ('free', 9), ('strong', 9), ('bullish', 9), ('great', 9), ('algory', 9), ('major', 9), ('least', 9), ('ltc', 8), ('fintech', 8), ('best', 7), ('trx', 7), ('real', 7), ('global', 7), ('ico', 7), ('arbitraj', 6), ('arbingtool', 6), ('ready', 6), ('eos', 6), ('short', 6), ('smart', 6), ('daily', 6), ('different', 6), ('few', 6), ('long', 6), ('total', 6), ('average', 6), ('much', 6), ('happy', 5), ('indian', 5), ('able', 5), ('bad', 5), ('own', 5), ('important', 5), ('bear', 5), ('several', 5), ('invest', 5), ('technical', 5), ('low', 5), ('less', 5), ('/', 5), ('tech', 5), ('ripple', 5), ('profitable', 4), ('>', 4), ('same', 4), ('ã¢â\x82¬â¦', 4), ('dead', 4), ('itã¢â\x82¬â\x84¢s', 4), ('higher', 4),

In [60]:
#commonly used words to train against
word_features = list(all_words.keys())[:5000]

save_word_features = open("pickled_algorithms/word_features.pickle","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()

In [61]:
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)
    
    return features

In [10]:
#print((find_features(movie_reviews.words("neg/cv000_29416.txt"))))

In [62]:
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [63]:
featuresets[1]

({'ï': False,
  'arbitraj': True,
  'arbingtool': True,
  '//': True,
  'btc': False,
  'googlechrome': False,
  'firefox': False,
  'bit.ly/btc-crypto-tab': False,
  'cool': False,
  'bitcoin': True,
  'late': False,
  'financial': False,
  'more': False,
  '300-earn': False,
  '500-earn': False,
  '1000-earn': False,
  '1500-earn': False,
  '2000-earn': False,
  'interested': False,
  '@': False,
  'interesting': False,
  'sure': False,
  'only': False,
  'current': False,
  'profitable': False,
  'most': False,
  'reputable': False,
  '>': False,
  'best': False,
  'linkedin.com/company/crypto': False,
  'happy': False,
  'first': False,
  'other': False,
  'bac5b4f2/': False,
  'parabolic': False,
  'ready': False,
  'same': False,
  'nu': False,
  'crypto': False,
  'ada': False,
  'bch': False,
  'eos': False,
  'eth': False,
  'ltc': False,
  'trx': False,
  'short': False,
  'sound': False,
  'free': False,
  'excellent': False,
  'alternative': False,
  'coindesk.com/coindesk-

save_featuresets = open("pickled_algorithms/featuresets.pickle","wb")
pickle.dump(featuresets, save_featuresets)
save_featuresets.close()

In [64]:
random.shuffle(featuresets)

In [66]:
training_set = featuresets[:600]
testing_set = featuresets[600:]

In [38]:
training_set

KeyboardInterrupt: 

In [67]:
# likelyhood = prior occurences x likihood / evidence

classifier = nltk.NaiveBayesClassifier.train(training_set)

#classifier_f = open("naivebayes.pickle", "rb")
#classifier = pickle.load(classifier_f)
#classifier_f.close()

print("Original Naive Bayes Algo accuracy:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(30)

Original Naive Bayes Algo accuracy: 55.21739130434783
Most Informative Features
                 digital = True              pos : neg    =      4.5 : 1.0
                     any = True              neg : pos    =      4.2 : 1.0
                    most = True              pos : neg    =      4.0 : 1.0
                   right = True              neg : pos    =      3.3 : 1.0
                    last = True              neg : pos    =      3.3 : 1.0
                      us = True              pos : neg    =      3.1 : 1.0
                  future = True              pos : neg    =      3.1 : 1.0
                     fun = True              pos : neg    =      3.1 : 1.0
                     ico = True              neg : pos    =      2.9 : 1.0
                    long = True              neg : pos    =      2.9 : 1.0
                     few = True              neg : pos    =      2.9 : 1.0
                  ripple = True              neg : pos    =      2.9 : 1.0
                    

In [68]:
# Mulitnominal classifier
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)
#MNB_classifier.show_most_informative_features(15)

MNB_classifier accuracy: 55.21739130434783


In [69]:
BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

BernoulliNB_classifier accuracy: 53.91304347826087


In [70]:
LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression_classifier accuracy: 51.73913043478261


In [71]:
SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

SGDClassifier_classifier accuracy: 52.60869565217391


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [72]:
SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC_classifier accuracy: 47.391304347826086


In [73]:
LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

LinearSVC_classifier accuracy: 49.56521739130435


In [74]:
NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

NuSVC_classifier accuracy: 50.8695652173913


## Vote classifier

In [75]:
voted_classifier = VoteClassifier(classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LinearSVC_classifier,
                                  NuSVC_classifier,
                                  SGDClassifier_classifier,
                                  LogisticRegression_classifier)

In [76]:
print("voted_classifier accuracy:", (nltk.classify.accuracy(voted_classifier,testing_set))*100)

voted_classifier accuracy: 53.47826086956522


In [98]:
for name, value in testing_set[5][0].items():
    if value == True:
        print(name)

bitcoin
ready
crypto
eos
eap
wine
altcoin
token
blockchain
https


In [77]:
print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])*100)

Classification: pos Confidence %: 57.14285714285714


In [79]:
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0])*100)

Classification: pos Confidence %: 57.14285714285714


In [80]:
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0])*100)

Classification: neg Confidence %: 100.0


In [81]:
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0])*100)

Classification: neg Confidence %: 71.42857142857143


In [82]:
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0])*100)

Classification: neg Confidence %: 85.71428571428571


In [83]:
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:",voted_classifier.confidence(testing_set[5][0])*100)

Classification: pos Confidence %: 100.0


In [ ]:
def sentiment(text):
    feats = find_features(text)
    
    return voted_classifier.classify(feats)


## Pickling, how to

save_classifier = open("naivebayes.pickle", "wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()